# Connect Four with Minimax AI

## Problem Statement

You are tasked with developing an AI for the classic **Connect Four** game, where two players alternate dropping pieces into a 6x7 grid. The goal is to get four consecutive pieces in a row, column, or diagonal. Your AI should use the **minimax algorithm** to determine the best moves **without implementing alpha-beta pruning**.

## Requirements

1. **Game Setup**: Implement a 6x7 grid for the Connect Four board. Each cell should either be empty or contain a piece from one of the two players (Player and AI).
   
2. **Piece Dropping**: Implement a function to drop a piece in the lowest available row within a chosen column. If the column is full, the move should be considered invalid.

3. **Winning Condition**: Develop a function to check if a player has won by getting four consecutive pieces in any row, column, or diagonal.

4. **Minimax Algorithm**:
   - Implement the **minimax algorithm** to evaluate all possible moves up to a given depth (e.g., 4).
   - The AI should aim to maximize its chances of winning while minimizing the player’s opportunities to win.
   - If a terminal state (win, loss, or draw) is reached at a particular depth, the algorithm should evaluate the board accordingly.
   
5. **Game Flow**:
   - The game should allow the player and AI to take alternate turns until one wins or the board is full (draw).
   - The player should be able to input a column to place their piece.
   - The AI should automatically choose a column based on the minimax evaluation.

6. **Display**: After each move, display the current board state to show piece placements.

## Objective

Design and implement the Connect Four game such that the AI uses the **minimax algorithm** to optimally decide its moves. The AI should play against the player and aim to win or force a draw by minimizing the player’s winning chances.


In [11]:
import math
import numpy as np

ROWS = 6
COLS = 7
PLAYER = 1  # User
AI = 2      # Computer
EMPTY = 0
WINDOW_LENGTH = 4

def create_board():
    return np.zeros((ROWS, COLS), dtype=int)

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROWS-1][col] == EMPTY

def get_next_open_row(board, col):
    for row in range(ROWS):
        if board[row][col] == EMPTY:
            return row

def print_board(board):
    print(np.flip(board, 0))

def winning_move(board, piece):
    for r in range(ROWS):
        for c in range(COLS - 3):
            if all(board[r][c+i] == piece for i in range(WINDOW_LENGTH)):
                return True
    for r in range(ROWS - 3):
        for c in range(COLS):
            if all(board[r+i][c] == piece for i in range(WINDOW_LENGTH)):
                return True
    for r in range(ROWS - 3):
        for c in range(COLS - 3):
            if all(board[r+i][c+i] == piece for i in range(WINDOW_LENGTH)):
                return True
    for r in range(3, ROWS):
        for c in range(COLS - 3):
            if all(board[r-i][c+i] == piece for i in range(WINDOW_LENGTH)):
                return True
    return False

def evaluate_window(board, piece):
    score = 0
    opp_piece = PLAYER if piece == AI else AI
    
    if board.count(piece) == 4:
        score += 100
    elif board.count(piece) == 3 and board.count(EMPTY) == 1:
        score += 5
    elif board.count(piece) == 2 and board.count(EMPTY) == 2:
        score += 2
    if board.count(opp_piece) == 3 and board.count(EMPTY) == 1:
        score -= 4
    
    return score

def score_position(board, piece):
    score = 0
    center_array = [board[r][COLS // 2] for r in range(ROWS)]
    score += center_array.count(piece) * 3
    return score

def is_terminal_node(board):
    return winning_move(board, PLAYER) or winning_move(board, AI) or len(get_valid_locations(board)) == 0

def minimax(board, depth, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    terminal = is_terminal_node(board)
    if depth == 0 or terminal:
        if terminal:
            if winning_move(board, AI):
                return (None, 1000000)
            elif winning_move(board, PLAYER):
                return (None, -1000000)
            else:  # Draw
                return (None, 0)
        else:
            return (None, score_position(board, AI))
    if maximizingPlayer:
        value=-math.inf
        column=np.random.choice(valid_locations)
        for col in valid_locations:
            temp = board.copy()
            row=get_next_open_row(board,col)
            drop_piece(temp,row,col,AI)
            new_score=minimax(temp,depth-1,False)[1]
            if new_score> value:
                value = new_score
                column=col
        return column,value
    else:
        value=math.inf
        column = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            temp_board = board.copy()
            drop_piece(temp_board, row, col, PLAYER)
            new_score = minimax(temp_board, depth-1, True)[1]
            if new_score < value:
                value = new_score
                column = col
        return column, value
            
def get_valid_locations(board):
    return [col for col in range(COLS) if is_valid_location(board, col)]

def best_move(board, depth=4):
    col, _ = minimax(board, depth, True)
    return col


# Game setup
board = create_board()
game_over = False
turn = 0  # Player goes first

## implement main function here
count=0
while not game_over:
    if turn==0:
        print("-------Player-------")
        col=int(input("Enter column (1-7): "))
        col-=1
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board,row,col,PLAYER)
            if winning_move(board, PLAYER):
                print("Player wins!")
                game_over = True
            
    else:
        print("---------AI---------")
        col = best_move(board)
        if is_valid_location(board, col):
            row = get_next_open_row(board, col)
            drop_piece(board, row, col, AI)
            print(f"AI chooses column {col}")
            if winning_move(board, AI):
                print("AI wins!")
                game_over = True
        
    turn = (turn+1)%2    
    print_board(board)
    
    if len(get_valid_locations(board)) == 0:
        print("It's a draw!")
        game_over = True

-------Player-------


[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0]]
---------AI---------
AI chooses column 3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [1 0 0 2 0 0 0]]
-------Player-------
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [1 0 0 2 0 0 0]]
---------AI---------
AI chooses column 3
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 1 0 0 0]
 [1 0 0 2 0 0 0]]
-------Player-------
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 2 0 0 0]
 [0 0 0 1 0 0 0]
 [1 0 0 2 0 0 0]]
---------AI---------
AI chooses column 0
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 2 0 0 0]
 [2 0 0 1 0 0 0]
 [1 0 0 2 0 0 0]]
-------Player-------
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 [0 0 0 2 0 0 0]
 [2 0 0 1 0 0 0]
 [1 0 1 2 0 0 0]]
---------AI---------
AI chooses column 0
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0]
 

IndexError: index 9 is out of bounds for axis 0 with size 7

In [ ]:
11